<a href="https://colab.research.google.com/github/Niel-77/Geminiagent/blob/main/Two_Agent_team_for_MechE_problems_using_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Creating two agent team using Gemini
This is the notebook for creating conversable agent using gemini api. In this notebook we will create a conversable agent to solve a simple mechanical engineering problem. We will be usingpython package, FEniCS65 as the simulation environment for solving elasticity problems using FEM.


## Creating Gemini API key
We	can obtain Gemini API key from https://aistudio.google.com/. Gemini 1.0, Gemini 1.5 Pro and Gemini 1.5 Flash can be used for free of charge but with limits set to each of them for Requests per minutes (RPM), tokens per minute (TPM) and requests per day (RPD).

## Preparing FeniCS for Colab
We will install the fenics package using the github link if it not installed already.

In [21]:
# try:
#     import dolfin
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
#     import dolfin

## Prepare autogen for collab

In [22]:
try:
    import chromadb
except ImportError:
    !pip install chromadb==0.4.15
    import chromadb

try:
    import tiktoken
except ImportError:
    !pip install tiktoken==0.5.1
    import tiktoken

try:
    import pypdf
except ImportError:
    !pip install pypdf==3.17.0
    import pypdf

## Install Autogen with gemini features

In [23]:
pip install pyautogen[gemini]

## Install the necessary dependencies

In [24]:

# Install pyautogen with the required extras
!pip install pyautogen[gemini,retrievechat,lmm] --use-deprecated=legacy-resolver


## Importing the necessary functions
(All the imported functions might not have been used)

In [25]:
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

## Creating configuration for gemini and setting API endpoint
The similar configuration model can be used for all LLM models. For using gpt along with gemini, for example, we can add another list of called *config_list_gpt* with the model name and api key. This might not be the best way to create configuration when using multiple agent team but this will help us to avoid OAI_CONFIG_LIST which I found to be a little bit confusing to use with colab.

In [26]:
from google.colab import userdata
geminiapi=userdata.get('GEMINIAPI')
config_list_gemini = [
     {
        "model": "gemini-pro",
        "api_key": geminiapi,
        "api_type": "google"
    }
]


In [27]:
gemini_config = {
    "seed": 25,  # change the seed for different trials
    "temperature": 0,
    "config_list": config_list_gemini,
    "request_timeout": 600,
    "retry_wait_time": 120,
}

## Creating assistant called engineer
Here we create a assistant called engineer with system message telling that they are a professional mechanical engineer with extensive background in python coding and FeniCS who specialize at creating error free python codes.

In [36]:
assistant = AssistantAgent(
    name="engineer",
    # system_message="""
    # You are a mechanical engineer with extensive background in python and FeniCS .\
    # You are a specialist in creating error free codes.\
    # You write code such that even an amateur can understand it easily.\
    # You say DONE when there are no errors left.
    # """,
    llm_config=gemini_config,
    max_consecutive_auto_reply=10)

user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "Mechanics",
        "use_docker": False,  # set to True or image name like "python:3" to use docker
    },
)


## Provide question to the engineer
The engineer receives the task description from the user.

In [41]:
user_proxy.initiate_chat(
    assistant,
    message="""A force of 12KN is acting vertically at free end in a cantilever beam of length 4m.\
    Take E=200 GPa and I= 0.36m^-4. Use two 1-D elements and find nodal displacements in the beam.""")

user_proxy (to engineer):

A force of 12KN is acting vertically at free end in a cantilever beam of length 4m.    Take E=200 GPa and I= 0.36m^-4. Use two 1-D elements and find nodal displacements in the beam.

--------------------------------------------------------------------------------
engineer (to user_proxy):

```python
# filename: cantilever_beam.py
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

# Define the material properties
E = 200e9  # Young's modulus in Pa
I = 0.36e-4  # Second moment of area in m^4

# Define the beam geometry
L = 4  # Length of the beam in m

# Define the force acting on the beam
F = 12e3  # Force in N

# Define the number of elements
n_elements = 2

# Define the element lengths
L_e = L / n_elements

# Define the element stiffness matrices
K_e = np.array([[12*E*I/L_e**3, 6*E*I/L_e**2, -12*E*I/L_e**3, 6*E*I/L_e**2],
                 [6*E*I/L_e**2, 4*E*I/L_e, -6*E*I/L_e**2, 2*E*I/L_e],
                 [-12*E

ChatResult(chat_id=None, chat_history=[{'content': 'A force of 12KN is acting vertically at free end in a cantilever beam of length 4m.    Take E=200 GPa and I= 0.36m^-4. Use two 1-D elements and find nodal displacements in the beam.', 'role': 'assistant'}, {'content': '```python\n# filename: cantilever_beam.py\nimport numpy as np\nfrom scipy.sparse import csr_matrix\nfrom scipy.sparse.linalg import spsolve\n\n# Define the material properties\nE = 200e9  # Young\'s modulus in Pa\nI = 0.36e-4  # Second moment of area in m^4\n\n# Define the beam geometry\nL = 4  # Length of the beam in m\n\n# Define the force acting on the beam\nF = 12e3  # Force in N\n\n# Define the number of elements\nn_elements = 2\n\n# Define the element lengths\nL_e = L / n_elements\n\n# Define the element stiffness matrices\nK_e = np.array([[12*E*I/L_e**3, 6*E*I/L_e**2, -12*E*I/L_e**3, 6*E*I/L_e**2],\n                 [6*E*I/L_e**2, 4*E*I/L_e, -6*E*I/L_e**2, 2*E*I/L_e],\n                 [-12*E*I/L_e**3, -6*E*I/L_e

In [42]:
user_proxy.send(
    recipient=assistant,
    message="""The global stiffness matrix is not 2 times the number of elements.
    It is 2 times the number of nodes in a given elemen. Here, 2 refers to displacement possible at one node.""",
)

user_proxy (to engineer):

The global stiffness matrix is not 2 times the number of elements. 
    It is 2 times the number of nodes in a given elemen. Here, 2 refers to displacement possible at one node.

--------------------------------------------------------------------------------
engineer (to user_proxy):

I apologize for the mistake in my previous response. I have corrected the code and it should now run without errors.

```python
# filename: cantilever_beam.py
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

# Define the material properties
E = 200e9  # Young's modulus in Pa
I = 0.36e-4  # Second moment of area in m^4

# Define the beam geometry
L = 4  # Length of the beam in m

# Define the force acting on the beam
F = 12e3  # Force in N

# Define the number of elements
n_elements = 2

# Define the element lengths
L_e = L / n_elements

# Define the element stiffness matrices
K_e = np.array([[12*E*I/L_e**3, 6*E*I/L_e**2, -12*E*I/L_